Name: Sang Heon Lee

ID: 20581451

Uwaterloo ID: sh68lee

In [1]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
%matplotlib inline
import numpy as np
from numpy.linalg import norm
from numpy import random
import pandas as pd
from scipy.stats import uniform, randint

#from scipy.stats import uniform, randint
from sklearn import svm
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb
import pickle

In [3]:
data_train = pd.read_csv('online_jewellery_shop_train.csv')

In [4]:
result_train = data_train.Revenue

In [5]:
result_train = result_train.to_frame()

In [6]:
data_train = data_train.drop('Revenue', axis=1)

In [7]:
data_train

,ID,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend
0,3935,1.0,50.333333,0.0,0.00,21.0,586.189286,0.000000,0.005556,0.0,0.6,May,1,1,1,2,Returning_Visitor,0
1,7866,6.0,299.000000,1.0,41.00,36.0,696.402778,0.000000,0.002632,0.0,0.0,May,2,2,2,4,New_Visitor,1
2,3727,0.0,0.000000,0.0,0.00,19.0,620.033333,0.000000,0.007895,0.0,0.0,Feb,1,1,4,2,Returning_Visitor,0
3,4454,0.0,0.000000,0.0,0.00,4.0,46.250000,0.050000,0.066667,0.0,0.0,Nov,3,2,6,13,Returning_Visitor,0
4,6186,0.0,0.000000,0.0,0.00,24.0,671.088653,0.026667,0.036566,0.0,0.0,Nov,1,1,3,2,Returning_Visitor,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8625,5935,0.0,0.000000,0.0,0.00,29.0,873.750000,0.000000,0.031034,0.0,0.0,Nov,2,2,5,2,Returning_Visitor,1
8626,7455,10.0,198.243056,3.0,276.25,151.0,4344.919986,0.002532,0.011966,0.0,0.0,Dec,4,2,4,2,Returning_Visitor,0
8627,7853,0.0,0.000000,0.0,0.00,7.0,431.533333,0.000000,0.007143,0.0,0.0,Oct,2,2,1,2,Returning_Visitor,0
8628,7355,2.0,16.000000,0.0,0.00,16.0,381.686508,0.011765,0.046569,0.0,0.6,Feb,2,4,2,1,Returning_Visitor,0


In [8]:
data_test = pd.read_csv('online_jewellery_shop_test_final.csv')

In [9]:
# Data cleaning of Object types to categorial data
data_train['VisitorType'] = data_train['VisitorType'].replace({'Returning_Visitor': 1, 'New_Visitor': 0, 'Other': 2})

In [10]:
# Data cleaning of Object types to categorical data
data_train['Month'] = data_train['Month'].replace({'Jan': 1, 'Feb': 2, 'Mar': 3, 'May': 5, 'June': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12})

In [11]:
# Cleaning of test data
data_test['VisitorType'] = data_test['VisitorType'].replace({'Returning_Visitor': 1, 'New_Visitor': 0, 'Other': 2})

In [12]:
# Cleaning of test data
data_test['Month'] = data_test['Month'].replace({'Jan': 1, 'Feb': 2, 'Mar': 3, 'May': 5, 'June': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12})

___

Basic XGB Classifier with no specified hyperparameter

In [13]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic")

In [14]:
xgb_model.fit(data_train, result_train.values.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
y_pred = xgb_model.predict_proba(data_test)

In [17]:
# function which outputs ID, Revenue predictions to csv

def makeResultCSV(ypred, i):
    y_pred_df = pd.DataFrame(ypred[:,1])
    y_pred_df.columns = ['Revenue']
    resultID = data_test.ID.to_frame()
    result = pd.concat([resultID, y_pred_df], join = 'outer', axis = 1) 
    return pd.DataFrame(result).to_csv("result" + str(i) + ".csv", index=False)

___

Define XGB Classifier and use train_test_split to split train data to train and test data

In [18]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric=["auc", "error"])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data_train, result_train, random_state=42)

In [20]:
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

[0]	validation_0-auc:0.91327	validation_0-error:0.11029
[1]	validation_0-auc:0.92240	validation_0-error:0.09731
[2]	validation_0-auc:0.92701	validation_0-error:0.09639
[3]	validation_0-auc:0.92663	validation_0-error:0.09870
[4]	validation_0-auc:0.92697	validation_0-error:0.10241
[5]	validation_0-auc:0.92725	validation_0-error:0.09546
[6]	validation_0-auc:0.92876	validation_0-error:0.09824
[7]	validation_0-auc:0.92769	validation_0-error:0.09917
[8]	validation_0-auc:0.92824	validation_0-error:0.09778
[9]	validation_0-auc:0.92878	validation_0-error:0.09685
[10]	validation_0-auc:0.92844	validation_0-error:0.09917
[11]	validation_0-auc:0.92764	validation_0-error:0.09731
[12]	validation_0-auc:0.92675	validation_0-error:0.09731
[13]	validation_0-auc:0.92759	validation_0-error:0.09824
[14]	validation_0-auc:0.92722	validation_0-error:0.09917
[15]	validation_0-auc:0.92678	validation_0-error:0.10148
[16]	validation_0-auc:0.92769	validation_0-error:0.10009
[17]	validation_0-auc:0.92738	validation_

C:\Users\sangh\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[33]	validation_0-auc:0.92715	validation_0-error:0.10287
[34]	validation_0-auc:0.92743	validation_0-error:0.10241
[35]	validation_0-auc:0.92709	validation_0-error:0.10658
[36]	validation_0-auc:0.92698	validation_0-error:0.10565
[37]	validation_0-auc:0.92707	validation_0-error:0.10241
[38]	validation_0-auc:0.92682	validation_0-error:0.10241
[39]	validation_0-auc:0.92652	validation_0-error:0.10241
[40]	validation_0-auc:0.92659	validation_0-error:0.10195
[41]	validation_0-auc:0.92603	validation_0-error:0.10334
[42]	validation_0-auc:0.92602	validation_0-error:0.10241
[43]	validation_0-auc:0.92611	validation_0-error:0.10195
[44]	validation_0-auc:0.92580	validation_0-error:0.09963
[45]	validation_0-auc:0.92620	validation_0-error:0.10241
[46]	validation_0-auc:0.92621	validation_0-error:0.10195
[47]	validation_0-auc:0.92586	validation_0-error:0.10102
[48]	validation_0-auc:0.92618	validation_0-error:0.10148
[49]	validation_0-auc:0.92640	validation_0-error:0.10241
[50]	validation_0-auc:0.92562	v

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1,
              eval_metric=['auc', 'error'], gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [23]:
y_pred_proba = xgb_model.predict_proba(data_test)
y_pred = xgb_model.predict(X_test)

roc_auc_score(y_test, y_pred)

0.7674559218135734

In [24]:
accuracy_score(y_test, y_pred)

0.8948100092678406

In [22]:
makeResultCSV(y_pred_proba, 2)

___

Hyperparameter searching with RandomizedSearchCV

In [70]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")



In [71]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", eval_metric=["auc", "error"])

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "min_child_weight" : uniform(0, 10),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 170), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, n_iter=300, cv=3, verbose=1, n_jobs=1, return_train_score=True)

In [72]:
search.fit(data_train, result_train.values.ravel())

Fitting 3 folds for each of 300 candidates, totalling 900 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 900 out of 900 | elapsed:  2.7min finished


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           eval_metric=['auc', 'error'],
                                           gamma=None, gpu_id=None,
                                           importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constr...
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000016B0536F850>,
        

In [73]:
report_best_scores(search.cv_results_, 3)

Model rank: 1
Mean validation score: 0.906 (std: 0.005)
Parameters: {'colsample_bytree': 0.8253661600623214, 'gamma': 0.3785096648408872, 'learning_rate': 0.051225222929687335, 'max_depth': 4, 'min_child_weight': 7.265118855950864, 'n_estimators': 125, 'subsample': 0.6855386943772857}

Model rank: 2
Mean validation score: 0.905 (std: 0.005)
Parameters: {'colsample_bytree': 0.8344553414554907, 'gamma': 0.4062120639203169, 'learning_rate': 0.04272015964853329, 'max_depth': 4, 'min_child_weight': 8.270600691923416, 'n_estimators': 133, 'subsample': 0.8198334353372777}

Model rank: 3
Mean validation score: 0.905 (std: 0.005)
Parameters: {'colsample_bytree': 0.7718944117410991, 'gamma': 0.3480690794486958, 'learning_rate': 0.11250845882446112, 'max_depth': 2, 'min_child_weight': 9.685220114763467, 'n_estimators': 112, 'subsample': 0.7816185350583484}



In [74]:
#result3 params
params1 = {
    "objective": "binary:logistic",
    "colsample_bytree": 0.7498850106268238,
    "gamma": 0.3690168082131852,
    "learning_rate": 0.054839600377537934, # default 0.1 
    "max_depth": 5, # default 3
    "n_estimators": 125, # default 100
    "subsample": 0.6272998821416366
}

In [75]:
#result4 params
params2 = {
    "objective": "binary:logistic",
    "colsample_bytree": 0.757395319248184,
    "gamma": 0.29039565691054975,
    "learning_rate": 0.03507706060836941, # default 0.1 
    "max_depth": 5, # default 3
    "n_estimators": 121, # default 100
    "subsample": 0.6078581298315237
}

In [76]:
#result5 params
params3 = {
    "colsample_bytree": 0.923680288120177,
    "min_child_weight" : 9.285210309624292,
    "gamma": 0.09421829851867058,
    "learning_rate": 0.0702898752019768, # default 0.1 
    "max_depth": 3, # default 3
    "n_estimators": 141, # default 100
    "subsample": 0.6834223814092886
}

In [77]:
#result6 params ***Gave BEST result***
params4 = {
    "colsample_bytree": 0.9792966574185944,
    "min_child_weight" : 6.195856405318516,
    "gamma": 0.27806516445616497,
    "learning_rate": 0.03932032935375214, # default 0.1 
    "max_depth": 4, # default 3
    "n_estimators": 103, # default 100
    "subsample": 0.6457264492878044
}

In [78]:
#results7 params
params5 = {
    "colsample_bytree": 0.9342606982013768,
    "min_child_weight" :6.173642520427691,
    "gamma": 0.3181897549880853,
    "learning_rate": 0.05110245551364666, # default 0.1 
    "max_depth": 4, # default 3
    "n_estimators": 168, # default 100
    "subsample": 0.8571126120422332
}

In [79]:
#results8 params
params6 = {
    "colsample_bytree": 0.9553970221768882,
    "min_child_weight": 6.835286879856128,
    "gamma": 0.34660348667718177,
    "learning_rate": 0.04971643162881245, # default 0.1 
    "max_depth": 4, # default 3
    "n_estimators": 188, # default 100
    "subsample": 0.7828998955388349
}

In [80]:
#results9 params
params7 = {
    "colsample_bytree": 0.9751124979381287,
    "min_child_weight": 5.689526844374235,
    "gamma": 0.40876385784491887,
    "learning_rate": 0.059280003046457294, # default 0.1 
    "max_depth": 5, # default 3
    "n_estimators": 113, # default 100
    "subsample": 0.7418771995156108
}

Apply the top hyperparameter to test data

In [81]:
xgb_model = xgb.XGBClassifier()
xgb_model.set_params(**params4)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9792966574185944,
              gamma=0.27806516445616497, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.03932032935375214,
              max_delta_step=None, max_depth=4,
              min_child_weight=6.195856405318516, missing=nan,
              monotone_constraints=None, n_estimators=103, n_jobs=None,
              num_parallel_tree=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None,
              subsample=0.6457264492878044, tree_method=None,
              validate_parameters=None, verbosity=None)

In [82]:
xgb_model.fit(data_train, result_train.values.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9792966574185944,
              gamma=0.27806516445616497, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.03932032935375214,
              max_delta_step=0, max_depth=4, min_child_weight=6.195856405318516,
              missing=nan, monotone_constraints='()', n_estimators=103,
              n_jobs=0, num_parallel_tree=1, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.6457264492878044,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [83]:
y_pred = xgb_model.predict_proba(data_test)

In [84]:
makeResultCSV(y_pred, 10)

This method gives the best result

___

Another method of searching Hyperparameter using GridSearchCV

In [168]:
clf = xgb.XGBClassifier(objective="binary:logistic")
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    #scoring="neg_log_loss",
                    cv=3)

grid.fit(data_train, result_train)

C:\Users\sangh\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, va

In [169]:
best_pars = grid.best_params_

In [170]:
best_pars

{'colsample_bytree': 0.5,
 'eta': 0.05,
 'gamma': 0.4,
 'max_depth': 4,
 'min_child_weight': 3}

In [171]:
best_model = grid.best_estimator_

In [172]:
best_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eta=0.05, gamma=0.4,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.0500000007, max_delta_step=0, max_depth=4,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [175]:
pickle.dump(grid.best_estimator_, open("xgb_log_reg.pickle", "wb"))

In [179]:
best_model_grid = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eta=0.05, gamma=0.4,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.0500000007, max_delta_step=0, max_depth=4,
              min_child_weight=3, missing='nan', monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [58]:
#best_model_grid.fit(data_train, result_train)